Import Library

In [618]:
import os
import numpy as np
from PIL import Image
import numpy as np
import math
from numpy.random import normal

Flatten

In [619]:
# Set the input directory path
input_directory = "../Dataset/Foto_Resize_70x70"

# List all folders in the input directory
folders = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
print(f"Found folders: {folders}")

# Initialize arrays for inputs and outputs
X = []
Y = []

# Create a mapping from folder names to one-hot encoded labels
label_map = {folder: idx for idx, folder in enumerate(folders)}
num_classes = len(folders)

# Process each folder and photo
for folder in folders:
    folder_path = os.path.join(input_directory, folder)
    
    # Sort the list of photos to ensure consistent order
    photos = sorted([p for p in os.listdir(folder_path) if p.endswith(('.png', '.jpg', '.jpeg'))])
    
    for photo in photos:
        photo_path = os.path.join(folder_path, photo)
        image = Image.open(photo_path)
        
        # Convert the image to a numpy array and flatten it
        image_array = np.array(image).flatten()
        X.append(image_array)
        
        # Create a one-hot encoded label
        label = np.zeros(num_classes)
        label[label_map[folder]] = 1
        Y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
X = X/255;
Y = np.array(Y)

print(f"Input array shape: {X.shape}")
print(f"Output array shape: {Y.shape}")

Found folders: ['Azmira', 'David', 'Dimas', 'Fadhli', 'Fadlin', 'Hafidz', 'Haidar', 'Hanna', 'Keiko', 'Khansa', 'Mikhael', 'Puti', 'Raesa', 'Satwika', 'Toni']
Input array shape: (346, 4900)
Output array shape: (346, 15)


In [620]:
print(Y)
np.shape(Y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


(346, 15)

Split Train & Test

In [621]:
import numpy as np

def split_data(X, Y, train_ratio, val_ratio):
  # Acak data
  num_samples = X.shape[0]  # Assume X and Y have the same number of samples
  indices = np.arange(num_samples)
  np.random.shuffle(indices)

  # Cari banyak data atau indeks yang akan digunakan untuk training dan validasi
  train_idx = int(train_ratio * num_samples)
  val_idx = int((train_ratio + val_ratio) * num_samples)

  # Split data
  X_train = X[indices[:train_idx]]
  Y_train = Y[indices[:train_idx]]
  X_val = X[indices[train_idx:val_idx]]
  Y_val = Y[indices[train_idx:val_idx]]
  X_test = X[indices[val_idx:]]
  Y_test = Y[indices[val_idx:]]

  # Kembalikan X Y untuk training, validasi, dan testing
  return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [622]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = split_data(X, Y, train_ratio=0.7, val_ratio=0.15)

Training

In [623]:
X = X_train
Y = Y_train

Initialize Layer

In [624]:
inputLayer = 4900
hiddenLayer = 1
outputLayer = 15

Calculating Weights for Input Layer to Hidden Layer

In [625]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 15
stddev = np.sqrt(2/n)
hiddenNeurons = 15
v = np.random.normal(loc=0, scale = stddev, size = (hiddenNeurons, inputLayer))
print(v)
np.shape(v)

[[-0.09124306  0.02481073 -0.57950529 ... -0.05801824 -0.22106324
   0.03072954]
 [ 0.31353267  0.06513772 -0.08641205 ...  0.31365074 -0.11995074
  -0.05123923]
 [-0.16057345 -0.73654059  0.70051094 ...  0.45830672  0.06491307
   0.29676461]
 ...
 [-0.5840363  -0.19352833 -0.34606319 ... -0.20997738 -0.0537327
  -0.06946391]
 [ 0.11784785 -0.37981277  0.24111245 ...  0.86664098  0.15375809
   0.05794724]
 [-0.1753335  -0.17459601  0.26137456 ... -0.73641686 -0.1085525
   0.20692505]]


(15, 4900)

In [626]:
vb = np.random.normal(loc=0, scale=stddev, size=hiddenNeurons) 
print(vb)
np.shape(vb)

[-0.08009173  0.19141364 -0.20482515  0.09986     0.24402833 -0.16719503
  0.06885085 -0.22847765 -0.22526595 -0.51505703 -0.50537234  0.08089643
  0.03271966  0.13646866  0.33310811]


(15,)

Calculating Weights for Hidden Layer to Output Layer

In [627]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 15
stddev = np.sqrt(2/n)
w = np.random.normal(loc=0, scale = stddev, size = (hiddenNeurons, outputLayer))
print(w)
np.shape(w)

[[-2.15938186e-01  1.80871588e-01 -4.00280551e-01  2.50286995e-01
   4.58148348e-01  5.47299812e-01  2.89481306e-01 -4.51253621e-02
   3.86207991e-01  7.34169033e-01  2.33437071e-03  4.06671270e-04
   4.75800749e-01  2.98249364e-01  7.27490949e-01 -3.17846157e-01
   3.06437286e-01 -6.23795186e-01  3.63762673e-01 -3.13924541e-01
  -5.77714567e-03  1.17903795e-01 -1.27632302e-01 -1.67962071e-01
  -1.38949087e-01 -4.35872052e-01 -1.51193699e-01  2.87077157e-01
   3.24699052e-01 -1.63854420e-01 -2.74914161e-01 -9.00311629e-02
   1.56952329e-01 -7.49159141e-01 -1.72756655e-01  7.18610455e-02
  -3.29705566e-01 -1.36069374e-01  4.60292246e-02 -1.96292490e-01
  -2.00235644e-01 -2.56408083e-01  5.70057228e-01  1.02771588e-01
   2.61928852e-01  7.92754662e-01 -3.16354597e-02 -4.24915203e-01
  -2.36848482e-01 -1.83131349e-01  1.10902475e-01 -1.08485258e-01]
 [ 2.67630783e-01 -4.97821598e-01  7.92069898e-01 -2.07926813e-01
  -1.37677618e-03 -1.16396660e-01  2.22647798e-01  9.98823007e-01
  -8.8503

(15, 52)

Calculating Bias Weight from Hidden Layer to Output Layer

In [628]:
wb = np.random.normal(loc=0, scale = stddev, size = outputLayer)
print(wb)
np.shape(wb)

[-0.35741446  0.36577506  0.20184747 -0.25274998  0.74474887 -0.37966559
 -0.39789293  0.5727562   0.07586184  0.28016214 -0.29994935  0.140899
 -0.03553598 -0.3992829  -0.06504005  0.68839887  0.24091797  0.11638338
 -0.4145008   0.40438095 -0.03259196  0.30837925  0.54330098  0.44405424
 -0.43566746  0.07236714  0.17564583  0.19928174 -0.23515581  0.71860912
  0.14903909 -0.31994501 -0.102255   -0.02451158  0.30274094  0.00133136
 -0.28869556 -0.14417541  0.15321337 -0.10356662 -0.03131877  0.36889432
  0.10722421 -0.24114415  0.64122126  0.11482091 -0.23937429 -0.03724968
 -0.42786687 -0.13397085  0.26290584  0.11079263]


(52,)

Feedforward

In [629]:
result = np.matmul(v, X.T)
z_in = result + vb
print(z_in)
np.shape(z_in)

ValueError: operands could not be broadcast together with shapes (15,242) (15,) 

In [585]:
def ReLU(x):
    return np.maximum(0, x)

In [586]:
z = ReLU(z_in)
print(z)
np.shape(z)

[[ 8.97911176 26.76078072 17.50122862 ... 25.43663191 19.57062023
  16.42215302]
 [ 0.          0.          0.         ...  0.84411584  0.
   0.        ]
 [ 0.          4.59209432  0.         ...  5.32358857  0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 9.99376623 14.90675756  3.34953417 ...  6.58861114  8.36996554
  10.22752198]
 [14.18083441  7.68964818 12.58756889 ... 17.4070245   6.49585751
   8.55262751]]


(15, 242)

In [587]:
#y_in = np.dot(z,w) + wb
y_in = np.dot(w, z) + wb[:, np.newaxis]
print(y_in)
np.shape(y_in)

[[ -5.54042378  -1.59698389  -8.0131055  ...   4.79282549  -8.06430664
   -2.65042922]
 [ -9.72967599 -15.57614539  -4.5496184  ...  -9.91617159 -14.14807738
  -12.91147045]
 [ 45.05524893  61.65465481  47.45818663 ...  56.50366741  47.35954076
   53.16623836]
 ...
 [ 37.08131675  26.07888974  34.06581624 ...  36.86368654  32.49883249
   32.40877567]
 [  0.73637268   9.13606938  -1.3984358  ...  -0.49250197  -1.14836105
    7.21993434]
 [ -5.12760289   0.44485045   6.58155255 ...   6.54708242  -4.46538182
   -4.24501227]]


(15, 242)

In [588]:
def Softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

In [589]:
y = Softmax(y_in)
print(y)
np.shape(y)

[[6.52809912e-32 3.36822369e-30 5.50699541e-33 ... 2.00659572e-27
  5.23212776e-33 1.17461470e-30]
 [9.89504162e-34 2.85975104e-36 1.75828006e-31 ... 8.21152113e-34
  1.19269742e-35 4.10754042e-35]
 [6.14056324e-10 9.93661777e-03 6.78876527e-09 ... 5.75695956e-05
  6.15105251e-09 2.04534306e-06]
 ...
 [2.11433337e-13 3.52273621e-18 1.03647357e-14 ... 1.70081808e-13
  2.16284718e-15 1.97658118e-15]
 [3.47348004e-29 1.54421134e-25 4.10798894e-30 ... 1.01641562e-29
  5.27515654e-30 2.27268821e-26]
 [9.86444320e-32 2.59512519e-29 1.20031206e-26 ... 1.15964213e-26
  1.91280867e-31 2.38438737e-31]]


(15, 242)

Back Propagation

In [590]:
sigma_k = y-Y.T
print(sigma_k)
np.shape(sigma_k)

[[ 6.52809912e-32  3.36822369e-30 -1.00000000e+00 ...  2.00659572e-27
   5.23212776e-33  1.17461470e-30]
 [ 9.89504162e-34  2.85975104e-36  1.75828006e-31 ...  8.21152113e-34
   1.19269742e-35  4.10754042e-35]
 [ 6.14056324e-10  9.93661777e-03  6.78876527e-09 ...  5.75695956e-05
   6.15105251e-09 -9.99997955e-01]
 ...
 [ 2.11433337e-13  3.52273621e-18  1.03647357e-14 ...  1.70081808e-13
   2.16284718e-15  1.97658118e-15]
 [ 3.47348004e-29  1.54421134e-25  4.10798894e-30 ...  1.01641562e-29
   5.27515654e-30  2.27268821e-26]
 [-1.00000000e+00  2.59512519e-29  1.20031206e-26 ...  1.15964213e-26
   1.91280867e-31  2.38438737e-31]]


(15, 242)

In [591]:
delta_w = (1/hiddenNeurons)*np.dot(sigma_k, z.T)
print(delta_w)



[[-1.55803384e+01 -7.36516421e-02 -1.64117754e-01 -5.53597216e+00
  -3.64136741e+00  4.66196671e-24 -1.76446338e+01 -7.54791678e+00
  -1.33725586e+01 -1.52222041e+01  0.00000000e+00 -2.97471012e+01
   8.87106794e-32 -3.27476420e+00 -9.87674390e+00]
 [-1.71671344e+01 -4.64442052e-01 -1.55449253e-01 -1.37240100e+01
  -9.07533257e+00  7.08233506e-33 -2.04459097e+01 -4.00467854e+00
  -2.04605856e+01 -8.98165119e+00  0.00000000e+00 -2.47415583e+01
   5.41987342e-36 -8.35042858e+00 -1.02288622e+01]
 [-1.10346440e+01 -5.85967274e-01 -7.69154898e-01 -1.30273671e+01
  -1.21542066e-01  1.75011532e-11 -1.79134939e+01 -5.11807621e+00
  -8.56779560e+00 -1.04500388e+01  0.00000000e+00 -2.29019647e+01
   2.47436862e-21 -5.21533223e+00 -4.44060211e+00]
 [-2.04559889e+01 -1.36331864e-01 -1.69891946e+00 -1.67617328e+01
  -5.63153800e+00  3.38962455e-25 -2.41010601e+01 -4.81979895e+00
  -2.42938939e+01 -9.62728161e+00  0.00000000e+00 -2.81869056e+01
   7.96085133e-33 -1.07948149e+01 -1.11002536e+01]
 [-2

In [592]:
delta_wb = (1/hiddenNeurons)*np.sum(sigma_k, axis=1, keepdims=True)
print(delta_wb)
np.shape(delta_wb)

[[-0.93333333]
 [-0.86666667]
 [-0.8       ]
 [-0.93333333]
 [-1.2       ]
 [-1.        ]
 [-1.06666667]
 [-1.        ]
 [-1.46666667]
 [-1.        ]
 [-0.93333333]
 [-1.13333333]
 [-1.2       ]
 [-0.73333333]
 [-1.8       ]]


(15, 1)

In [593]:
def d_ReLU(x):
    return x>0

In [594]:
sigma_in = np.dot(wb.T, sigma_k)*d_ReLU(z_in)
print(sigma_in)
np.shape(sigma_in)

[[ 0.15764287 -0.08356885  0.49828892 ...  0.02003261  0.13904458
   0.52506083]
 [ 0.         -0.          0.         ...  0.02003261  0.
   0.        ]
 [ 0.         -0.08356885  0.         ...  0.02003261  0.
   0.        ]
 ...
 [ 0.         -0.          0.         ...  0.          0.
   0.        ]
 [ 0.15764287 -0.08356885  0.49828892 ...  0.02003261  0.13904458
   0.52506083]
 [ 0.15764287 -0.08356885  0.49828892 ...  0.02003261  0.13904458
   0.52506083]]


(15, 242)

In [ ]:
delta_v = (1/hiddenNeurons)*np.dot(sigma_in, X)
print(delta_v)
np.shape(delta_v)

[[2.75867356e+00 2.75567260e+00 2.75552941e+00 ... 1.79011290e+00
  2.01581277e+00 2.08686460e+00]
 [5.11966894e-01 5.09720222e-01 5.10224178e-01 ... 2.73801031e-01
  2.74424088e-01 2.77771069e-01]
 [5.08406802e-01 5.08507452e-01 5.08149196e-01 ... 3.73770463e-01
  3.83641076e-01 3.93241363e-01]
 ...
 [1.54945045e-02 1.54945045e-02 1.50757341e-02 ... 3.87362613e-03
  2.93139275e-03 2.40792976e-03]
 [2.65323332e+00 2.65092177e+00 2.65066399e+00 ... 1.68428886e+00
  1.91200135e+00 1.98529840e+00]
 [2.72926003e+00 2.72591174e+00 2.72615097e+00 ... 1.77249811e+00
  1.99796242e+00 2.06921346e+00]]


(15, 4900)

In [596]:
delta_vb = (1/hiddenNeurons)*np.sum(sigma_in, axis=1, keepdims=True)
print(delta_vb)

[[3.35529401]
 [0.58900379]
 [0.61377577]
 [3.38199062]
 [2.2689902 ]
 [0.26174269]
 [3.35529401]
 [3.25153698]
 [3.38199062]
 [3.30190078]
 [0.        ]
 [3.38199062]
 [0.02669661]
 [3.23313785]
 [3.3108488 ]]


Update Parameters

In [597]:
alpha = 0.2
w = w - alpha*delta_w
print(w)
wb = wb - alpha*delta_wb
print(wb)
v = v - alpha*delta_v
print(v)
vb = vb- alpha*delta_vb
print(vb)

[[ 3.19612274e+00  2.03736667e-01 -2.44358389e-01  2.14537484e+00
   8.11525960e-01  4.80743617e-01  3.22896058e+00  1.07894503e+00
   2.47246397e+00  3.61094243e+00  4.83944447e-02  5.57193491e+00
  -8.76835893e-02  1.07299487e-01  1.81952772e+00]
 [ 3.52930607e+00 -2.75631375e-01  3.67000566e-01  2.14194698e+00
   1.63373074e+00  3.56232482e-01  3.96312672e+00  7.07879131e-01
   3.87970930e+00  1.75622343e+00  2.64235247e-01  4.88618438e+00
   3.37616227e-01  1.52606842e+00  2.71118576e+00]
 [ 2.46111252e+00  1.80866978e-01  2.35168916e-01  3.00579712e+00
   1.98273981e-01 -3.90659274e-01  3.67681091e+00  1.32705950e+00
   2.00995664e+00  2.41733187e+00  1.88470843e-01  5.24228274e+00
  -4.21272902e-01  1.45968142e+00  1.23590471e+00]
 [ 4.23533376e+00  9.54096150e-02  3.06765561e-01  4.14187576e+00
   1.04254964e+00 -6.31731148e-01  4.86176299e+00  5.13142605e-02
   5.11287712e+00  1.36841137e+00 -4.75123651e-02  5.20695831e+00
  -9.03330428e-02  1.98195099e+00  3.08559585e+00]
 [ 4

In [598]:
error_quadratic = 0.5*np.sum((Y_train.T-y)**2)
print(error_quadratic)

121.36303978661579


Validasi

In [599]:
X = X_val
Y = Y_val